## 1. Import modules & Data

In [1]:
import pandas as pd 
import sys
import openpyxl
import string

In [2]:
# Load the workbook
excel_file = openpyxl.load_workbook('/Users/yue/Documents/Plasmid_crosstalk_modeling-/raw_data/RNAcurvehigherconc.xlsx', data_only=True)

# Get the worksheets 
Gain70_worksheet = excel_file['Gain 70']
Gain75_worksheet = excel_file['Gain 75']
raw_data_worksheet = excel_file['Plate 1 - Sheet1']

# Load workbooks using pandas 
degradation_data_updated_gain70 = pd.read_excel('/Users/yue/Documents/Plasmid_crosstalk_modeling-/raw_data/RNAcurvehigherconc.xlsx',sheet_name = 'Gain 70')
degradation_data_updated_gain75 = pd.read_excel('/Users/yue/Documents/Plasmid_crosstalk_modeling-/raw_data/RNAcurvehigherconc.xlsx',sheet_name = 'Gain 75')

## 2. Extract data and organize into time-course for individual replicates

### 2.0 Define universal variables

In [3]:
conc_vec = [10,50,100,500,1000,5000,10000,20000]

### 2.1 Replace timestamp with timeVec

In [4]:
def get_timeVec(df):
    timeVec = df['Time'].apply(lambda x:(x.hour*60+x.minute)*60+x.second)
    return timeVec

In [5]:
timeVec_gain70 = get_timeVec(degradation_data_updated_gain70)
timeVec_gain75 = get_timeVec(degradation_data_updated_gain75)

### 2.2 Find raw data from the other worksheet 

**Find the last row**

In [6]:
# Initialize the last row variable
last_row = 0

# Loop through the rows in reverse order
for row in reversed(range(1, Gain70_worksheet.max_row + 1)):
    for cell in Gain70_worksheet[row]:
        if cell.value is not None and str(cell.value).strip() != "":
            last_row = row
            break
    if last_row != 0:
        break

print(f"The last non-empty row is: {last_row}")

The last non-empty row is: 47


In [7]:
start_row_idx = 2
end_row_idx = last_row + 1 

**Hardcode columns and start/end row for the two gains**

In [8]:
raw_data_columns_gain_70 = []
raw_data_columns_gain_75 = []
raw_data_row_gain_70_start_idx = []
raw_data_row_gain_70_end_idx = []
raw_data_row_gain_75_start_idx = []
raw_data_row_gain_75_end_idx = [] 

# Get column names for gain 70 & 75
start_1 = string.ascii_uppercase.index('A')
end_1 = string.ascii_uppercase.index('R') + 1
for letter in string.ascii_uppercase[start_1:end_1]:
        # Column name
    column_string = 'C' + letter
    raw_data_columns_gain_70.append(column_string)
    raw_data_columns_gain_75.append(column_string)
        # Row start and end indices 
    raw_data_row_gain_70_start_idx.append(136)
    raw_data_row_gain_70_end_idx.append(181)
    raw_data_row_gain_75_start_idx.append(336)
    raw_data_row_gain_75_end_idx.append(381)

start_2 = string.ascii_uppercase.index('G')
end_2 = string.ascii_uppercase.index('L') + 1
for letter in string.ascii_uppercase[start_2:end_2]:
    raw_data_columns_gain_70.append(letter)
    raw_data_row_gain_70_start_idx.append(186)
    raw_data_row_gain_70_end_idx.append(231)
    
start_2 = string.ascii_uppercase.index('G')
end_2 = string.ascii_uppercase.index('I') + 1
for letter in string.ascii_uppercase[start_2:end_2]:
    raw_data_columns_gain_75.append(letter)
    raw_data_row_gain_75_start_idx.append(386)
    raw_data_row_gain_75_end_idx.append(431)


**Organize column names**

In [9]:
column_name_list_gain_70 = []
column_name_list_gain_75 = []
for conc in conc_vec:
    for rep in range(1,4):
        column_name_list_gain_70.append(str(conc) + '_' + str(rep))

for conc in conc_vec[:-1]:
    for rep in range(1,4):
        column_name_list_gain_75.append(str(conc) + '_' + str(rep))

#### 2.3 Compile raw data into dataframe

In [10]:

# # Create a dictionary to hold the data for each column
# data = {col: [] for col in column_name_list_gain_70}


# # Loop through and retrieve data
# for idx,col in enumerate(raw_data_columns_gain_70):
#     start_row = raw_data_row_gain_70_start_idx[idx]
#     end_row = raw_data_row_gain_70_end_idx[idx]
#     col_name = column_name_list_gain_70[idx]
#     bg_row_idx = 2
#     for row in range(start_row, end_row + 1):
#         cell_value = raw_data_worksheet[f'{col}{row}'].value
#         bg_value = Gain70_worksheet[f'K{bg_row_idx}'].value
#         bg_row_idx += 1
#         # Subtract background 
#         data[col_name].append(cell_value - bg_value)


In [11]:

# Create a dictionary to hold the data for each column
data = {col: [] for col in column_name_list_gain_75}

# Loop through and retrieve data
for idx,col in enumerate(raw_data_columns_gain_75):
    start_row = raw_data_row_gain_75_start_idx[idx]
    end_row = raw_data_row_gain_75_end_idx[idx]
    col_name = column_name_list_gain_75[idx]
    bg_row_idx = 2
    for row in range(start_row, end_row + 1):
        cell_value = raw_data_worksheet[f'{col}{row}'].value
        bg_value = Gain70_worksheet[f'J{bg_row_idx}'].value
        bg_row_idx += 1
        # Subtract background 
        data[col_name].append(cell_value - bg_value)

TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'

In [ ]:
df = pd.DataFrame(data)

In [ ]:
output_file_name_updated70 = '/Users/yue/Documents/Plasmid_crosstalk_modeling-/processed_data/degradation_curve_gain_75.xlsx'
with pd.ExcelWriter(output_file_name_updated75, mode='a', engine='openpyxl') as writer:  
        df.to_excel(writer,sheet_name='single_sample_deg')

In [ ]:
df